In [2]:
# Imports
import xpress as xp
import networkx as nx

# NetworkX tutorial available here: 
# https://networkx.org/documentation/stable/tutorial.html

In [3]:
# Parameters for the optimization:
k = 3 # max cycle length

In [4]:
# # Example input data 1

# pairs = ["P1", "P2", "P3", "P4"] 
# altruistic_donors = ["NDD1"]
# nodes = pairs + altruistic_donors
# edges = {("NDD1", "P1"): 2,
#          ("P1", "P2"): 10, 
#          ("P2", "P3"): 10,
#          ("P3", "P4"): 10,
#          ("P4", "P1"): 10
# }

# Example input data 2

pairs = ["P1", "P2", "P3", "P4", "P5"] 
altruistic_donors = ["NDD1"]
nodes = pairs + altruistic_donors
edges = {("NDD1", "P1"): 0.1,
         ("P1", "P2"): 10, 
         ("P2", "P3"): 9,
         ("P3", "P4"): 8,
         ("P4", "P5"): 7,
         ("P5", "P1"): 6
}

## Example input data 3

# pairs = ["P1", "P2", "P3", "P4", "P5"] 
# altruistic_donors = ["NDD1"]
# nodes = pairs + altruistic_donors
# edges = {("NDD1", "P1"): 0.1,
#          ("P1", "P2"): 10, 
#          ("P2", "P3"): 9,
#          ("P3", "P4"): 8,
#          ("P4", "P5"): 7
# }

In [5]:
# Create the loop!

# Create Xpress Model
# Initialize the model
prob = xp.problem()

# Define decision variables for each edge
y = {e: xp.var(vartype=xp.binary, name=f"y_{e[0]}_{e[1]}") for e in edges}
prob.addVariable(list(y.values()))

# Objective: Maximize total benefit
prob.setObjective(xp.Sum(y[e] * w for e, w in edges.items()), sense=xp.maximize)

# Constraints
for v in pairs:
    prob.addConstraint(xp.Sum(y[e] for e in edges if e[0] == v) <= xp.Sum(y[e] for e in edges if e[1] == v))
    prob.addConstraint(xp.Sum(y[e] for e in edges if e[1] == v) <= 1)

for a in altruistic_donors:
    prob.addConstraint(xp.Sum(y[e] for e in edges if e[0] == a) <= 1)


# Add a test callback - COULDN'T WORK OUT HOW TO MAKE WORK QUICKLY ENOUGH
# prob.addcbpreintsol(myCallback,None,1)

finished = False # A flag to mark the end of the optimization.
infeasible = False # A (currently unused) flag to mark no feasible solution.
# TODO: Add a catch for no feasible solution. This shouldn't happen but it is
# probably better to be robust about this.

while finished == False and infeasible == False:
  
    # Solve the model
    prob.solve()
    opt_sol = prob.getSolution()

    # Construct the graph from the optimal solution:
    DG = nx.DiGraph()
    selected_edges = [list(edges.keys())[i] for i, e in enumerate(list(edges.keys())) if opt_sol[i]==1]
    DG.add_edges_from(selected_edges)

    # Check if there is a cycle length that is too long:
    cycles = list(nx.simple_cycles(DG))

    # If ok, report done.
    # TODO: Rewrite this so that it is with the max_cycle bit...
    if cycles==[] or max(map(len,cycles))<=k:
        print("")
        print("##########################################################")
        print(f"OPTIMIZATION COMPLETE: no cycles of length more than {k}.")
        print("##########################################################")
        print("")
        finished = True
        break
    
    # If not done, report that reoptimization is required:
    else:
        print("")
        print("#################################################################")
        print(f"REOPTIMIZATION REQUIRED: proposed solution contains long cycles.")
        print("#################################################################")
        print("")

    # Take the long cycle we found and make a note of its edges:
    max_cycle = max(cycles,key=len)
    cycle_edges = [(max_cycle[i],max_cycle[i+1]) for i in range(len(max_cycle)-1)]
    cycle_edges += [(max_cycle[-1],max_cycle[0])]

    # Add the constraint to remove this as an option: 
    prob.addConstraint(xp.Sum(y[e] for e in cycle_edges) <= len(max_cycle)-1)


# Print the output
print("")
print("")
print("")

print("Optimal Matches:")
for (u, v), var in y.items():
    if prob.getSolution(var) > 0.5:
        print(f"{u} donates to {v} with benefit {edges[(u,v)]}")

print(f"Total Benefit: {prob.getObjVal()}")
print("")
print("")
print("")

FICO Xpress v9.4.2, Hyper, solve started 12:28:11, Feb 19, 2025
Heap usage: 391KB (peak 391KB, 85KB system)
Maximizing MILP noname using up to 16 threads and up to 15GB memory, with these control settings:
OUTPUTLOG = 1
NLPPOSTSOLVE = 1
XSLP_DELETIONCONTROL = 0
XSLP_OBJSENSE = -1
Original problem has:
        11 rows            6 cols           18 elements         6 entities
Presolved problem has:
         0 rows            0 cols            0 elements         0 entities
Presolve finished in 0 seconds
Heap usage: 395KB (peak 406KB, 85KB system)
Will try to keep branch and bound tree memory usage below 8.9GB
Starting concurrent solve with dual (1 thread)

 Concurrent-Solve,   0s
            Dual        
    objective   dual inf
 D  40.000000   .0000000
                        
------- optimal --------
Concurrent statistics:
           Dual: 0 simplex iterations, 0.00s
Optimal solution found
 
   Its         Obj Value      S   Ninf  Nneg   Sum Dual Inf  Time
     0         40.000000     

C:\Users\ajmac\AppData\Local\Temp\ipykernel_37972\3555581866.py:5: LicenseWarning: Using the license file found in your Xpress installation. If you want to use this license and no longer want to see this message, use the following code before using the xpress module:
  xpress.init('C:/Program Files/xpressmp/bin/xpauth.xpr')

  prob = xp.problem()
